In [ ]:
import os
import json
import hashlib
import getpass   # ✅ for hiding password input

USERS_FILE = "users.json"
TASKS_DIR = "tasks"


# ---------------------------------------------
# Helper: Hash password for security
# ---------------------------------------------
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()


# ---------------------------------------------
# 1. User Authentication (Registration & Login)
# ---------------------------------------------
def load_users():
    if not os.path.exists(USERS_FILE):
        return {}
    with open(USERS_FILE, "r") as f:
        return json.load(f)


def save_users(users):
    with open(USERS_FILE, "w") as f:
        json.dump(users, f, indent=4)


def register():
    print("\n--- User Registration ---")
    users = load_users()
    username = input("Enter a username: ").strip()

    if username in users:
        print("Username already exists. Please choose another.\n")
        return None

    # ✅ Hide password input
    password = getpass.getpass("Enter a password: ").strip()
    confirm = getpass.getpass("Confirm password: ").strip()

    if password != confirm:
        print("Passwords do not match. Try again.\n")
        return

    users[username] = hash_password(password)
    save_users(users)
    print("Registration successful! You can now log in.\n")


def login():
    print("\n--- User Login ---")
    users = load_users()
    username = input("Enter username: ").strip()
    # ✅ Hide password input
    password = getpass.getpass("Enter password: ").strip()

    if username in users and users[username] == hash_password(password):
        print(f"Welcome, {username}!\n")
        return username
    else:
        print("Invalid credentials. Try again.\n")
        return None


# ---------------------------------------------
# 2. Load and Save Tasks per User
# ---------------------------------------------
def get_task_file(username):
    if not os.path.exists(TASKS_DIR):
        os.makedirs(TASKS_DIR)
    return os.path.join(TASKS_DIR, f"{username}_tasks.json")


def load_tasks(username):
    file_path = get_task_file(username)
    if not os.path.exists(file_path):
        return []
    with open(file_path, "r") as f:
        return json.load(f)


def save_tasks(username, tasks):
    file_path = get_task_file(username)
    with open(file_path, "w") as f:
        json.dump(tasks, f, indent=4)


# ---------------------------------------------
# 3. Add a Task
# ---------------------------------------------
def add_task(username):
    print("\n--- Add New Task ---")
    description = input("Enter task description: ").strip()
    tasks = load_tasks(username)
    task_id = len(tasks) + 1

    task = {
        "id": task_id,
        "description": description,
        "status": "Pending"
    }
    tasks.append(task)
    save_tasks(username, tasks)
    print("Task added successfully!\n")


# ---------------------------------------------
# 4. View Tasks
# ---------------------------------------------
def view_tasks(username):
    print("\n--- Your Tasks ---")
    tasks = load_tasks(username)
    if not tasks:
        print("No tasks found.\n")
        return

    for task in tasks:
        print(f"ID: {task['id']} | Description: {task['description']} | Status: {task['status']}")
    print()


# ---------------------------------------------
# 5. Mark Task as Completed
# ---------------------------------------------
def mark_task_completed(username):
    print("\n--- Mark Task as Completed ---")
    tasks = load_tasks(username)
    if not tasks:
        print("No tasks available.\n")
        return

    try:
        task_id = int(input("Enter task ID to mark as completed: "))
    except ValueError:
        print("Invalid input! Enter a numeric ID.\n")
        return

    for task in tasks:
        if task["id"] == task_id:
            task["status"] = "Completed"
            save_tasks(username, tasks)
            print("Task marked as completed!\n")
            return
    print("Task ID not found.\n")


# ---------------------------------------------
# 6. Delete a Task
# ---------------------------------------------
def delete_task(username):
    print("\n--- Delete a Task ---")
    tasks = load_tasks(username)
    if not tasks:
        print("No tasks available.\n")
        return

    try:
        task_id = int(input("Enter task ID to delete: "))
    except ValueError:
        print("Invalid input! Enter a numeric ID.\n")
        return

    tasks = [t for t in tasks if t["id"] != task_id]

    # Reassign IDs
    for idx, task in enumerate(tasks, start=1):
        task["id"] = idx

    save_tasks(username, tasks)
    print("Task deleted successfully!\n")


# ---------------------------------------------
# 7. Menu after Login
# ---------------------------------------------
def task_menu(username):
    while True:
        print(f"--- Task Manager ({username}) ---")
        print("1. Add Task")
        print("2. View Tasks")
        print("3. Mark Task as Completed")
        print("4. Delete Task")
        print("5. Logout")

        choice = input("Enter your choice: ").strip()

        if choice == "1":
            add_task(username)
        elif choice == "2":
            view_tasks(username)
        elif choice == "3":
            mark_task_completed(username)
        elif choice == "4":
            delete_task(username)
        elif choice == "5":
            print("Logging out...\n")
            break
        else:
            print("Invalid choice. Try again.\n")


# ---------------------------------------------
# 8. Main Menu (Login / Register / Exit)
# ---------------------------------------------
def main():
    while True:
        print("=== Task Manager System ===")
        print("1. Register")
        print("2. Login")
        print("3. Exit")

        choice = input("Enter your choice: ").strip()

        if choice == "1":
            register()
        elif choice == "2":
            user = login()
            if user:
                task_menu(user)
        elif choice == "3":
            print("Goodbye!")
            break
        else:
            print("Invalid option. Try again.\n")


# Run Program
if __name__ == "__main__":
    main()

=== Task Manager System ===
1. Register
2. Login
3. Exit


Enter your choice:  2



--- User Login ---


Enter username:  immaculate
Enter password:  ········


Welcome, immaculate!

--- Task Manager (immaculate) ---
1. Add Task
2. View Tasks
3. Mark Task as Completed
4. Delete Task
5. Logout


...


--- Mark Task as Completed ---


Enter task ID to mark as completed:  1


Task marked as completed!

--- Task Manager (immaculate) ---
1. Add Task
2. View Tasks
3. Mark Task as Completed
4. Delete Task
5. Logout


Enter your choice:  2



--- Your Tasks ---
ID: 1 | Description: Self learning | Status: Completed

--- Task Manager (immaculate) ---
1. Add Task
2. View Tasks
3. Mark Task as Completed
4. Delete Task
5. Logout
